# TextAttack

In [ ]:
!pip install emoji texthero flair gdown 
!pip install textattack==0.3.0

  Using cached importlib_metadata-3.10.1-py3-none-any.whl (14 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.8.2:
      Successfully uninstalled importlib-metadata-4.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


In [ ]:
!pip install tensorflow_text

  Using cached importlib_metadata-4.8.2-py3-none-any.whl (17 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
konoha 4.6.5 requires importlib-metadata<4.0.0,>=3.7.0, but you have importlib-metadata 4.8.2 which is incompatible.


In [ ]:
import pandas as pd
import re 
import emoji
import texthero as hero
from texthero import preprocessing
import zipfile
import os
import random
import gdown
from textattack.attack_recipes import DeepWordBugGao2018, PSOZang2020, TextFoolerJin2019, BAEGarg2019, CheckList2020
from textattack.loggers.csv_logger import CSVLogger
from textattack.datasets import Dataset
from flair.models import TextClassifier
import time
from flair.data import Sentence
from pandas import DataFrame
from textattack.models.wrappers import ModelWrapper

# A custom model wrapper 
* This Class is instantiated with the path to a trained model 
* An instance of this class can be called with a list of texts and it returns a list of predictions 

In [9]:
class CustomModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model
    
    def __call__(self, text_input_list):
        df = DataFrame(text_input_list,columns=['claimReview_claimReviewed'])
        sentences = [Sentence(sentence) for sentence in df['claimReview_claimReviewed'].to_list()]
        self.model.predict(sentences, verbose = False, mini_batch_size=32)
        predictions=[]
        for sentence in sentences:
            score = sentence.labels[0].score
            if sentence.labels[0].value == 'TRUE':
                predictions.append([1-score,score])
            else:
                predictions.append([score, 1-score])
        return predictions

# Create a custom_dataset of false claims to be used to attack the trained model 

In [13]:
!wget https://github.com/untruenews/ss2021/raw/main/data/data.csv.zip
zf = zipfile.ZipFile("data.csv.zip") 
# open
df = pd.read_csv(zf.open('data_out.csv')).sample(n=3000, random_state=1)
df["rating_alternateName_normalized"]=df["rating_alternateName_normalized"].str.upper()

# taking only the english text
train_df=df[df.language=="en"][["claimReview_claimReviewed","rating_alternateName_normalized"]]
custom_dataset = []
for index, row in train_df[train_df['rating_alternateName_normalized']=='FALSE'].iterrows():
    custom_dataset.append((row['claimReview_claimReviewed'], 0))

--2021-12-03 03:05:53--  https://github.com/untruenews/ss2021/raw/main/data/data.csv.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/untruenews/ss2021/main/data/data.csv.zip [following]
--2021-12-03 03:05:53--  https://raw.githubusercontent.com/untruenews/ss2021/main/data/data.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85485669 (82M) [application/zip]
Saving to: ‘data.csv.zip.1’

data.csv.zip.1      100%[===================>]  81.53M   223MB/s    in 0.4s    

2021-12-03 03:06:57 (223 MB/s) - ‘data.csv.zip.1’ saved [85485669/85485669]



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.



In [14]:
random.seed(44)
custom_dataset= random.sample(custom_dataset,500)

# Load Pretrained Models & Attack

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir "/content/drive/MyDrive/text-attack-results"

techniques = ['DeepWordBugGao2018', 'PSOZang2020', 'TextFoolerJin2019', 'BAEGarg2019', 'CheckList2020']
models = ['flair','berttweet']


for model_name in models:
  for recipe in techniques:
    print(recipe)
    print(model_name)

    recept_name = recipe
    timestr = time.strftime("%Y%m%d-%H%M%S")
    result_dir = "/content/drive/MyDrive/text-attack-results/results-" + recept_name +"-" + timestr
    os.mkdir(result_dir)

    if model_name == 'flair':
      url = 'https://drive.google.com/uc?id=1xh7O-Wl4Rwr-eau1OrBQRCRXvZ6Z31J4' 
    else:
      url = 'https://drive.google.com/uc?id=1m1Zqif7HH4alCoEPBKwXk-9gXXuP0mDh' 
    output = 'best-model.pt' 
    gdown.download(url, output, quiet = True)

    model = TextClassifier.load('./best-model.pt') 
    model_wrapper = CustomModelWrapper(model)

    if recipe == 'DeepWordBugGao2018':  
      attack =  DeepWordBugGao2018.build(model_wrapper);
    elif recipe == 'PSOZang2020':  
      attack =  PSOZang2020.build(model_wrapper);
    elif recipe == 'TextFoolerJin2019':  
      attack =  TextFoolerJin2019.build(model_wrapper);
    elif recipe == 'BAEGarg2019':  
      attack =  BAEGarg2019.build(model_wrapper);
    elif recipe == 'CheckList2020':  
      attack =  CheckList2020.build(model_wrapper);
    
    dataset = Dataset(custom_dataset)

    # Add timestamp to result file
    logger = CSVLogger(color_method='html', filename=result_dir+'/results-'+model_name+'.csv')

    for example, label in custom_dataset:
      result = attack.attack(example, label)
      logger.log_attack_result(result)
      print(result.__str__(color_method='ansi'))

    # Write the result csv to google drive
    logger.flush()
    summary = logger.df['result_type'].value_counts()
    summary.to_csv(result_dir+'/summary-'+model_name+'.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/MyDrive/text-attack-results’: File exists
DeepWordBugGao2018
flair
2021-12-03 03:14:52,881 loading file ./best-model.pt


textattack: Unknown if model of class <class 'flair.models.text_classification_model.TextClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/text-attack-results/results-DeepWordBugGao2018-20211203-031449/results-flair.csv


0 (91%) --> [FAILED]

A photograph shows boxer Muhammad Ali with musician Michael Jackson.
PSOZang2020
flair
2021-12-03 03:15:11,640 loading file ./best-model.pt


textattack: Downloading https://textattack.s3.amazonaws.com/transformations/hownet/word_candidates_sense.pkl.
100%|██████████| 8.39M/8.39M [00:00<00:00, 12.8MB/s]
textattack: Copying /root/.cache/textattack/tmpbk20knvc.zip to /root/.cache/textattack/transformations/hownet/word_candidates_sense.pkl.
textattack: Successfully saved transformations/hownet/word_candidates_sense.pkl to cache.
textattack: CSVLogger exiting without calling flush().
textattack: CSVLogger exiting without calling flush().
textattack: Unknown if model of class <class 'flair.models.text_classification_model.TextClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/text-attack-results/results-PSOZang2020-20211203-031507/results-flair.csv


2021-12-03 03:15:14,904 --------------------------------------------------------------------------------
2021-12-03 03:15:14,906 The model key 'upos-fast' now maps to 'https://huggingface.co/flair/upos-english-fast' on the HuggingFace ModelHub
2021-12-03 03:15:14,912  - The most current version of the model is automatically downloaded from there.
2021-12-03 03:15:14,914  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/upos-fast/en-upos-ontonotes-fast-v0.4.pt)
2021-12-03 03:15:14,919 --------------------------------------------------------------------------------


...

textattack: Unknown if model of class <class 'flair.models.text_classification_model.TextClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/text-attack-results/results-PSOZang2020-20211203-032206/results-berttweet.csv


0 (100%) --> 1 (100%)

A photograph shows boxer Muhammad Ali with musician Michael Jackson.

A stereo conveys packer Muhammad Ali with organist Michael Jackson.
TextFoolerJin2019
berttweet
2021-12-03 03:23:01,496 loading file ./best-model.pt


textattack: Unknown if model of class <class 'flair.models.text_classification_model.TextClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/text-attack-results/results-TextFoolerJin2019-20211203-032244/results-berttweet.csv


0 (100%) --> 1 (100%)

A photograph shows boxer Muhammad Ali with musician Michael Jackson.

A photograph spectacle boxer Mehmet Ali with musician Macha Brandon.
BAEGarg2019
berttweet
2021-12-03 03:23:23,676 loading file ./best-model.pt


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'flair.models.text_classification_model.TextClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/text-attack-results/results-BAEGarg2019-20211203-032315/results-berttweet.

# Attack results 
* The results can be either `Successful`, `Failed` or `Skipped`
* An attack is skipped when the original text is missclassified by the model and therefore the text won't be used to attack the model

In [ ]:
berttweet_summary = pd.read_csv("/content/drive/MyDrive/results/summary-berttweet.csv")
berttweet_summary

In [ ]:
flair_summary = pd.read_csv("/content/drive/MyDrive/results/summary-flair.csv")
flair_summary